In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
import os
from typing import List, Dict, Any

In [6]:
df = pd.read_csv("D:\Atharva engineering\PROJECTS\SkillSync all data\Datasets for SkillSync\job_recommendation_dataset.csv")
df.head()


,Job Title,Company,Location,Experience Level,Salary,Industry,Required Skills
0,Early years teacher,Richardson Ltd,Sydney,Senior Level,87000.0,Healthcare,Pharmaceuticals
1,Counselling psychologist,"Ramos, Santiago and Stewart",San Francisco,Mid Level,50000.0,Marketing,"Google Ads, SEO, Content Writing"
2,Radio broadcast assistant,Franco Group,New York,Mid Level,77000.0,Healthcare,"Patient Care, Nursing, Medical Research, Pharm..."
3,"Designer, exhibition/display",Collins Inc,Berlin,Senior Level,90000.0,Software,Machine Learning
4,"Psychotherapist, dance movement",Barker Group,Sydney,Entry Level,112000.0,Healthcare,"Nursing, Medical Research, Pharmaceuticals"


In [9]:
# Data Cleaning
df['skills_list'] = df['Required Skills'].apply(
    lambda x: [s.strip().lower() for s in str(x).split(',')]
)
df.head()

,Job Title,Company,Location,Experience Level,Salary,Industry,Required Skills,skills_list
0,Early years teacher,Richardson Ltd,Sydney,Senior Level,87000.0,Healthcare,Pharmaceuticals,[pharmaceuticals]
1,Counselling psychologist,"Ramos, Santiago and Stewart",San Francisco,Mid Level,50000.0,Marketing,"Google Ads, SEO, Content Writing","[google ads, seo, content writing]"
2,Radio broadcast assistant,Franco Group,New York,Mid Level,77000.0,Healthcare,"Patient Care, Nursing, Medical Research, Pharm...","[patient care, nursing, medical research, phar..."
3,"Designer, exhibition/display",Collins Inc,Berlin,Senior Level,90000.0,Software,Machine Learning,[machine learning]
4,"Psychotherapist, dance movement",Barker Group,Sydney,Entry Level,112000.0,Healthcare,"Nursing, Medical Research, Pharmaceuticals","[nursing, medical research, pharmaceuticals]"


In [17]:
from scipy.sparse import hstack   
from sklearn.preprocessing import normalize


text_corpus = df['Job Title'] + " " + df['Required Skills']
df.tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
text_features = df.tfidf_vectorizer.fit_transform(text_corpus)
# Skill Encoding
skill_mlb = MultiLabelBinarizer()
skill_features = skill_mlb.fit_transform(df['skills_list'])

# Combine safely (SPARSE)
combined = hstack([text_features, skill_features])

# Normalize (SPARSE-SAFE)
combined = normalize(combined, norm='l2')